In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
from random import randrange, sample
from tqdm import tqdm

df = pd.read_csv('steam-200k.csv')
df.columns = ['id','game','state','hours_played','0']
df['id'].nunique()
df = df.drop('0',axis=1)
df.isnull().values.any()
bought_games = df.loc[df['state']=='purchase']
played_games = df.loc[df['state']=='play']
played_games_orig = played_games.drop(['state'],axis=1)


We want to see how well we can recommend games using frequent itemset

In [2]:
grouped = played_games_orig.drop('hours_played', axis=1)
grouped = grouped.groupby('id')['game'].apply(list).reset_index(name='games')
grouped

id                                              games
0           5250  [Cities Skylines, Deus Ex Human Revolution, Po...
1          76767  [Counter-Strike, Call of Duty World at War, To...
2          86540  [The Elder Scrolls V Skyrim, Audiosurf, XCOM E...
3         144736                                   [Counter-Strike]
4         181212           [Counter-Strike, Half-Life 2 Lost Coast]
...          ...                                                ...
11345  309434439                                           [Dota 2]
11346  309554670                                [Mitos.is The Game]
11347  309626088                     [Age of Empires II HD Edition]
11348  309824202                                           [Dota 2]
11349  309903146                                           [Dota 2]

[11350 rows x 2 columns]

In [3]:

te = TransactionEncoder()
te_ary = te.fit(grouped['games']).transform(grouped['games'])
df = pd.DataFrame(te_ary, columns=te.columns_)
df
fis = fpgrowth(df, min_support=0.005, use_colnames=True)
fis = fis.sort_values(by=['support'], ascending=False)
fis

support                                           itemsets
0     0.426520                                           (Dota 2)
1     0.204670                                  (Team Fortress 2)
6     0.121322                  (Counter-Strike Global Offensive)
39    0.094185                                         (Unturned)
20    0.070573                                    (Left 4 Dead 2)
...        ...                                                ...
2128  0.005022  (Sid Meier's Civilization V, Team Fortress 2, ...
2933  0.005022  (The Elder Scrolls V Skyrim, Don't Starve Toge...
2940  0.005022  (Team Fortress 2, Terraria, Don't Starve Toget...
2941  0.005022  (Garry's Mod, Team Fortress 2, Don't Starve To...
2507  0.005022  (Left 4 Dead 2, The Elder Scrolls V Skyrim, Co...

[3224 rows x 2 columns]

In [4]:
ar = association_rules(fis, metric="confidence", min_threshold=0.2)
ar.sort_values(by=['confidence', 'lift'], ascending=False)

antecedents  \
4746   (Counter-Strike Condition Zero Deleted Scenes,...   
1867      (Counter-Strike Condition Zero Deleted Scenes)   
7370             (Call of Duty Modern Warfare 2, Dota 2)   
10871  (Counter-Strike Global Offensive, Team Fortres...   
12547                (Insurgency, Counter-Strike Source)   
...                                                  ...   
3693                                          (PAYDAY 2)   
3641                                          (PAYDAY 2)   
3728                                          (PAYDAY 2)   
4792                                  (Unturned, Dota 2)   
6680                                 (Heroes & Generals)   

                                             consequents  antecedent support  \
4746                                    (Counter-Strike)            0.006432   
1867                                    (Counter-Strike)            0.008282   
7370       (Call of Duty Modern Warfare 2 - Multiplayer)            0.005815   
10871      (Call of Duty Modern Warfare 2 - Multiplayer)            0.005374   
12547                  (Counter-Strike Global Offensive)            0.005198   
...                                                  ...                 ...   
3693   (Counter-Strike Global Offensive, Team Fortres...            0.034361   
3641         (The Elder Scrolls V Skyrim, Left 4 Dead 2)            0.034361   
3728                 (Unturned, Team Fortress 2, Dota 2)            0.034361   
4792       (Garry's Mod, Team Fortress 2, Left 4 Dead 2)            0.032159   
6680                               (Unturned, Robocraft)            0.029515   

       consequent support   support  confidence       lift  leverage  \
4746             0.050044  0.006432    1.000000  19.982394  0.006110   
1867             0.050044  0.008194    0.989362  19.769816  0.007779   
7370             0.025639  0.005727    0.984848  38.412475  0.005578   
10871            0.025639  0.005286    0.983607  38.364036  0.005149   
12547            0.121322  0.005110    0.983051   8.102852  0.004479   
...                   ...       ...         ...        ...       ...   
3693             0.017004  0.006872    0.200000  11.761658  0.006288   
3641             0.020441  0.006872    0.200000   9.784483  0.006170   
3728             0.022555  0.006872    0.200000   8.867188  0.006097   
4792             0.019295  0.006432    0.200000  10.365297  0.005811   
6680             0.020969  0.005903    0.200000   9.537815  0.005284   

       conviction  
4746          inf  
1867    89.295859  
7370    64.307841  
10871   59.436035  
12547   51.842026  
...           ...  
3693     1.228744  
3641     1.224449  
3728     1.221806  
4792     1.225881  
6680     1.223789  

[13560 rows x 9 columns]

In [5]:
def recommendation_ar(librairy, fis, ar):
    item_set_tmp = frozenset()

    for i in range(ar.shape[0]):
        if( set(ar.iloc[i][0]).issubset(set(librairy))):
            #print("antecedent : " + str(list(ar.iloc[i][0])) + " | consequent : " + str(list(ar.iloc[i][1])) + " | confidence : " + str(ar.iloc[i][5]))
            item_set_tmp = frozenset.union(item_set_tmp,ar.iloc[i][1])
    #if(one):
    #    for i in range(fis.shape[0]):
    #       if(set(librairy).issubset(set(fis.iloc[i][1]))):
    #            item_set_tmp = frozenset.union(item_set_tmp,fis.iloc[i][1])
    item_set = list(frozenset.difference(item_set_tmp, librairy))
    return item_set

In [6]:
librairy = ['Cities Skylines']
print(librairy)
print(recommendation_ar(librairy, fis, ar))

for i in range (10):
    librairy = grouped.iloc[i][1]
    print(librairy)
    print(recommendation_ar(librairy, fis, ar))
    print('\n')

['Cities Skylines']
["Sid Meier's Civilization V"]
['Cities Skylines', 'Deus Ex Human Revolution', 'Portal 2', 'Alien Swarm', 'Team Fortress 2', 'Dota 2']
['Counter-Strike Source', 'The Binding of Isaac', 'Portal', "Sid Meier's Civilization V", 'Just Cause 2', 'Left 4 Dead 2', "Garry's Mod", 'PAYDAY 2', 'Grand Theft Auto IV', 'Unturned', 'Half-Life 2', 'Dishonored', 'Castle Crashers', 'Magicka', 'Tomb Raider', 'Fallout New Vegas', 'Left 4 Dead', "Mirror's Edge", 'Warframe', 'PAYDAY The Heist', 'Metro 2033', 'The Elder Scrolls V Skyrim', 'Chivalry Medieval Warfare', 'Killing Floor', 'Far Cry 3', 'Dead Island', 'Borderlands 2', 'Counter-Strike Global Offensive', 'Saints Row The Third', 'XCOM Enemy Unknown', 'Terraria', 'BioShock Infinite', 'Torchlight II', 'BioShock', 'Trine 2']


['Counter-Strike', 'Call of Duty World at War', 'Total War ATTILA', 'Call of Duty Modern Warfare 2 - Multiplayer', 'Call of Duty Modern Warfare 2', 'Counter-Strike Source', 'Banished', 'Call of Duty Black Ops',

On peut remarquer que pour des supports allant de 10% à 5% on obtient presque uniquement les jeux les plus joués. Ce qui n'est pas étonnant comme on n'a pas filtré selon les temps de jeux. Obtenir une recommendation par association semble donc être futile.
On va voir s'il est possible de faire .
On va déjà faire rentrer le temps de jeu en paramètre : Si un joueur n'a pas plus d'heure de jeu que la moyenne pour un certain jeu, on ne le prend pas en compte.

In [7]:
avg = played_games_orig.groupby('game')['hours_played'].apply(np.average).reset_index(name='avg_hours_played')
avg

game  avg_hours_played
0                                           007 Legends          0.700000
1                                             0RBITALIS          0.400000
2     1... 2... 3... KICK IT! (Drop That Beat Like a...          4.000000
3                                       10 Second Ninja          2.950000
4                                            10,000,000          3.600000
...                                                 ...               ...
3595                                         rymdkapsel          1.100000
3596                                       sZone-Online          0.977586
3597                          the static speaks my name          0.250000
3598                                          theHunter          2.493548
3599                                   theHunter Primal         21.475000

[3600 rows x 2 columns]

In [8]:
#Dictionnary of average time played
avg_dict = avg.set_index('game').to_dict()['avg_hours_played']
avg_dict

{'007 Legends': 0.7,
 '0RBITALIS': 0.39999999999999997,
 '1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)': 4.0,
 '10 Second Ninja': 2.95,
 '10,000,000': 3.6,
 '100% Orange Juice': 8.700000000000001,
 '1000 Amps': 0.1,
 '12 Labours of Hercules': 4.957142857142856,
 '12 Labours of Hercules II The Cretan Bull': 4.2875,
 '12 Labours of Hercules III Girl Power': 6.533333333333334,
 '140': 1.26,
 '15 Days': 0.5,
 '16bit Trader': 4.45,
 '1701 A.D. Sunken Dragon': 0.6,
 '18 Wheels of Steel American Long Haul': 5.8,
 '1953 NATO vs Warsaw Pact': 5.9,
 '1Quest': 2.7,
 '3 Stars of Destiny': 4.7,
 '3089 -- Futuristic Action RPG': 0.3,
 '3D Mini Golf': 1.0,
 '3DMark': 13.377777777777778,
 '3DMark 11': 3.0,
 '3DMark Vantage': 21.0,
 '4 Elements': 14.85,
 '404Sight': 0.5285714285714286,
 '500 Years Act 1': 3.0,
 '60 Seconds!': 1.86,
 '7 Days to Die': 41.586734693877546,
 '8BitBoy': 3.4,
 '8BitMMO': 0.4555555555555555,
 '9 Clues 2 The Ward': 5.1,
 '9 Clues The Secret of Serpent Creek': 2.8,

In [9]:
#To get the avg time played for a game
avg_dict['Dota 2']

202.78549886387108

In [10]:
#Select only the rows where players played more than average
played_games_sup_avg = played_games_orig.loc[played_games_orig['hours_played'] > played_games_orig['game'].astype(str).map(avg_dict)]
played_games_sup_avg.sort_values(by='id')

id                                         game  hours_played
65423        5250                              Cities Skylines         144.0
65425        5250                     Deus Ex Human Revolution          62.0
55918       76767                             Total War ATTILA         207.0
55920       76767  Call of Duty Modern Warfare 2 - Multiplayer         165.0
55922       76767                Call of Duty Modern Warfare 2          65.0
...           ...                                          ...           ...
150248  307631446            MEDIEVAL Total War - Gold Edition          14.2
177078  307688442                                    Hurtworld          65.0
96358   307701164                                Emily is Away           0.9
117507  309255941                            Mitos.is The Game           5.7
169504  309554670                            Mitos.is The Game           5.9

[18331 rows x 3 columns]

In [11]:
g = played_games_sup_avg.drop('hours_played', axis=1)
g = g.groupby('id')['game'].apply(list).reset_index(name='games')
print(g)
te2 = TransactionEncoder()
te_ary2 = te2.fit(g['games']).transform(g['games'])
df2 = pd.DataFrame(te_ary2, columns=te2.columns_)
fis2 = fpgrowth(df2, min_support=0.001, use_colnames=True)
fis2 = fis2.sort_values(by=['support'], ascending=False)
fis2

             id                                              games
0          5250        [Cities Skylines, Deus Ex Human Revolution]
1         76767  [Counter-Strike, Call of Duty World at War, To...
2         86540  [The Elder Scrolls V Skyrim, Audiosurf, Killer...
3        229911  [Counter-Strike Condition Zero, Call of Duty M...
4        298950  [Team Fortress 2, Counter-Strike Global Offens...
...         ...                                                ...
4237  307631446                [MEDIEVAL Total War - Gold Edition]
4238  307688442                                        [Hurtworld]
4239  307701164                                    [Emily is Away]
4240  309255941                                [Mitos.is The Game]
4241  309554670                                [Mitos.is The Game]

[4242 rows x 2 columns]


support                                           itemsets
123   0.202263                                           (Dota 2)
11    0.093352                  (Counter-Strike Global Offensive)
12    0.081094                                  (Team Fortress 2)
478   0.059642                                         (Unturned)
8     0.049033                       (The Elder Scrolls V Skyrim)
...        ...                                                ...
1590  0.001179                   (Portal 2, Alien Swarm, Magicka)
2422  0.001179          (PAYDAY 2, Middle-earth Shadow of Mordor)
2421  0.001179  (ARK Survival Evolved, Middle-earth Shadow of ...
984   0.001179  (Call of Duty Modern Warfare 2 - Multiplayer, ...
1792  0.001179  (Borderlands 2, Crysis 2 Maximum Edition, BioS...

[3584 rows x 2 columns]

In [12]:
ar2 = association_rules(fis2, metric="confidence", min_threshold=0.2)
ar2 = ar2.sort_values(by=['confidence'], ascending=False)
ar2 

antecedents  \
3149              (Half-Life 2, Mass Effect)   
1326   (Empire Total War, Crusader Kings II)   
1320   (Empire Total War, Crusader Kings II)   
942     (Half-Life, Half-Life 2 Episode One)   
2471  (FTL Faster Than Light, Saints Row IV)   
...                                      ...   
1411          (The Witcher Enhanced Edition)   
1407          (The Witcher Enhanced Edition)   
1403          (The Witcher Enhanced Edition)   
1399          (The Witcher Enhanced Edition)   
2700                 (Half-Life 2, Portal 2)   

                                            consequents  antecedent support  \
3149                                    (Mass Effect 2)            0.001179   
1326                            (Mount & Blade Warband)            0.001414   
1320        (Mount & Blade Warband, Total War SHOGUN 2)            0.001414   
942                                       (Half-Life 2)            0.001650   
2471                                (Fallout New Vegas)            0.001179   
...                                                 ...                 ...   
1411  (Tomb Raider, The Witcher 2 Assassins of Kings...            0.007072   
1407  (Portal, The Witcher 2 Assassins of Kings Enha...            0.007072   
1403  (The Elder Scrolls V Skyrim, The Witcher 2 Ass...            0.007072   
1399                                          (Magicka)            0.007072   
2700                                (Half-Life, Portal)            0.005893   

      consequent support   support  confidence        lift  leverage  \
3149            0.004243  0.001179         1.0  235.666667  0.001174   
1326            0.005422  0.001414         1.0  184.434783  0.001407   
1320            0.002357  0.001414         1.0  424.200000  0.001411   
942             0.029939  0.001650         1.0   33.401575  0.001601   
2471            0.022159  0.001179         1.0   45.127660  0.001153   
...                  ...       ...         ...         ...       ...   
1411            0.002357  0.001414         0.2   84.840000  0.001398   
1407            0.001886  0.001414         0.2  106.050000  0.001401   
1403            0.004243  0.001414         0.2   47.133333  0.001384   
1399            0.015323  0.001414         0.2   13.052308  0.001306   
2700            0.002357  0.001179         0.2   84.840000  0.001165   

      conviction  
3149         inf  
1326         inf  
1320         inf  
942          inf  
2471         inf  
...          ...  
1411    1.247053  
1407    1.247643  
1403    1.244696  
1399    1.230846  
2700    1.247053  

[3463 rows x 9 columns]

In [13]:
librairy = ['Cities Skylines']
print(librairy)
print(recommendation_ar(librairy, fis2, ar2))

for i in range (10):
    librairy = g.iloc[i][1]
    print(librairy)
    print(recommendation_ar(librairy, fis2, ar2))
    print('\n')

['Cities Skylines']
['Prison Architect']
['Cities Skylines', 'Deus Ex Human Revolution']
['Half-Life 2', 'Dishonored', 'Prison Architect', 'Metro 2033', 'The Elder Scrolls V Skyrim', 'Team Fortress 2', 'Fallout New Vegas']


['Counter-Strike', 'Call of Duty World at War', 'Total War ATTILA', 'Call of Duty Modern Warfare 2 - Multiplayer', 'Call of Duty Modern Warfare 2', 'Call of Duty Black Ops']
['PAYDAY 2', 'Call of Duty Modern Warfare 3', 'Counter-Strike Global Offensive', 'Team Fortress 2', 'Call of Duty Black Ops - Multiplayer', 'Call of Duty Modern Warfare 3 - Multiplayer', 'Total War SHOGUN 2', 'Empire Total War', 'Total War ROME II - Emperor Edition']


['The Elder Scrolls V Skyrim', 'Audiosurf', 'Killer is Dead']
['Team Fortress 2']


['Counter-Strike Condition Zero', 'Call of Duty Modern Warfare 2']
['Call of Duty Black Ops', 'Call of Duty Modern Warfare 2 - Multiplayer', 'Counter-Strike', 'Call of Duty Modern Warfare 3']


['Team Fortress 2', 'Counter-Strike Global Offensive'

Let's evaluate our model

In [14]:
geval = g
print(geval)
geval = geval.sample(frac=1, random_state=42)
train_nb = int(geval.shape[0] / 100 * 80)
print(train_nb)
X_train = geval[:train_nb]
X_test = geval[train_nb:]
print(X_train)
X_test

             id                                              games
0          5250        [Cities Skylines, Deus Ex Human Revolution]
1         76767  [Counter-Strike, Call of Duty World at War, To...
2         86540  [The Elder Scrolls V Skyrim, Audiosurf, Killer...
3        229911  [Counter-Strike Condition Zero, Call of Duty M...
4        298950  [Team Fortress 2, Counter-Strike Global Offens...
...         ...                                                ...
4237  307631446                [MEDIEVAL Total War - Gold Edition]
4238  307688442                                        [Hurtworld]
4239  307701164                                    [Emily is Away]
4240  309255941                                [Mitos.is The Game]
4241  309554670                                [Mitos.is The Game]

[4242 rows x 2 columns]
3393
             id                                              games
3296  196113123                                        [Far Cry 2]
4179  296260221  [Resident Evil 

id                                              games
3540  209804474                             [No More Room in Hell]
2724  161816812  [Counter-Strike Global Offensive, Dota 2, Garr...
3044  181549899                                      [Dead Island]
3094  183801719                                           [Dota 2]
1460   95586525  [Call of Duty Modern Warfare 3 - Multiplayer, ...
...         ...                                                ...
3444  203712998                            [Football Manager 2015]
466    44240866  [Call of Duty Modern Warfare 2 - Multiplayer, ...
3092  183574382                                       [Dino D-Day]
3772  233532285                               [Grand Theft Auto V]
860    64971477                                          [Loadout]

[849 rows x 2 columns]

In [15]:
te3 = TransactionEncoder()
te_ary3 = te3.fit(X_train['games']).transform(X_train['games'])
dftrain = pd.DataFrame(te_ary3, columns=te3.columns_)
fistrain = fpgrowth(dftrain, min_support=0.001, use_colnames=True)
fistrain = fistrain.sort_values(by=['support'], ascending=False)
fistrain

support                                           itemsets
2     0.206012                                           (Dota 2)
3     0.091954                  (Counter-Strike Global Offensive)
56    0.078691                                  (Team Fortress 2)
119   0.058650                                         (Unturned)
7     0.048630                       (The Elder Scrolls V Skyrim)
...        ...                                                ...
1369  0.001179                  (Prison Architect, Hotline Miami)
1370  0.001179       (Deus Ex Human Revolution, Prison Architect)
1371  0.001179           (Prison Architect, Saints Row The Third)
2754  0.001179  (The Elder Scrolls V Skyrim, Mass Effect 2, Fa...
4179  0.001179                                 (Solar 2, Magicka)

[4180 rows x 2 columns]

In [16]:
artrain = association_rules(fistrain, metric="confidence", min_threshold=0.2)
artrain = artrain.sort_values(by=['antecedent support'], ascending=False)
artrain

antecedents  \
0                     (Counter-Strike Global Offensive)   
1                                       (Left 4 Dead 2)   
2                                         (Garry's Mod)   
4                                         (Garry's Mod)   
14                                        (Garry's Mod)   
...                                                 ...   
2535  (Total War ROME II - Emperor Edition, PAYDAY 2...   
2537   (PAYDAY 2, Total War SHOGUN 2, Total War ATTILA)   
4257                (Sniper Elite V2, PAYDAY The Heist)   
2541                       (PAYDAY 2, Total War ATTILA)   
4922                                          (Solar 2)   

                                            consequents  antecedent support  \
0                                              (Dota 2)            0.091954   
1                                     (Team Fortress 2)            0.038904   
2                     (Counter-Strike Global Offensive)            0.035367   
4                                     (Team Fortress 2)            0.035367   
14                                           (Unturned)            0.035367   
...                                                 ...                 ...   
2535                                 (Total War ATTILA)            0.001179   
2537              (Total War ROME II - Emperor Edition)            0.001179   
4257                                    (Left 4 Dead 2)            0.001179   
2541  (Total War ROME II - Emperor Edition, Total Wa...            0.001179   
4922                                          (Magicka)            0.001179   

      consequent support   support  confidence        lift  leverage  \
0               0.206012  0.020041    0.217949    1.057940  0.001098   
1               0.078691  0.012084    0.310606    3.947140  0.009022   
2               0.091954  0.010315    0.291667    3.171875  0.007063   
4               0.078691  0.009726    0.275000    3.494663  0.006943   
14              0.058650  0.007663    0.216667    3.694221  0.005589   
...                  ...       ...         ...         ...       ...   
2535            0.002947  0.001179    1.000000  339.300000  0.001175   
2537            0.009431  0.001179    1.000000  106.031250  0.001168   
4257            0.038904  0.001179    1.000000   25.704545  0.001133   
2541            0.005010  0.001179    1.000000  199.588235  0.001173   
4922            0.015915  0.001179    1.000000   62.833333  0.001160   

      conviction  
0       1.015263  
1       1.336404  
2       1.281947  
4       1.270770  
14      1.201723  
...          ...  
2535         inf  
2537         inf  
4257         inf  
2541         inf  
4922         inf  

[4923 rows x 9 columns]

In [17]:

X_test_formatted = X_test

def remove_n_random_items(lst, n):
    if(len(lst)>1):
        to_delete = set(sample(range(len(lst)), n))

        return [
            item for index, item in enumerate(lst)
            if not index in to_delete
        ]


X_test_formatted['incomplet_games'] = X_test_formatted['games'].apply(lambda x : remove_n_random_items(x,1))
X_test_formatted = X_test_formatted.dropna()
print(X_test_formatted)



             id                                              games  \
2724  161816812  [Counter-Strike Global Offensive, Dota 2, Garr...   
1460   95586525  [Call of Duty Modern Warfare 3 - Multiplayer, ...   
2818  167013261                            [Portal, Dragon's Lair]   
1893  117022366  [Euro Truck Simulator 2, Call of Duty 4 Modern...   
2000  122620165                [Dota 2, Sir, You Are Being Hunted]   
...         ...                                                ...   
769    60217594  [Counter-Strike, Counter-Strike Global Offensi...   
1685  106834867  [Team Fortress 2, Portal 2, Half-Life, Half-Li...   
130    14906179  [Counter-Strike Source, Half-Life 2 Deathmatch...   
3171  188681523  [Counter-Strike Global Offensive, PAYDAY 2, In...   
466    44240866  [Call of Duty Modern Warfare 2 - Multiplayer, ...   

                                        incomplet_games  
2724  [Counter-Strike Global Offensive, Dota 2, Garr...  
1460                    [Call of Duty Moder

/tmp/ipykernel_7755/2656098971.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_formatted['incomplet_games'] = X_test_formatted['games'].apply(lambda x : remove_n_random_items(x,1))


In [18]:
true_pos = 0
false_pos = 0

for i in tqdm(range(X_test_formatted.shape[0])):
    item_removed = set(X_test_formatted.iloc[i][1]).difference(set(X_test_formatted.iloc[i][2]))
    recommended_games = recommendation_ar(X_test_formatted.iloc[i][2],fistrain,artrain)
    if(item_removed.issubset(set(recommended_games))):
        true_pos = true_pos + 1
    else :
        false_pos = false_pos +1

precision = true_pos/(true_pos+false_pos)
print(true_pos)
print(false_pos)
print(precision)

100%|██████████| 447/447 [02:31<00:00,  2.95it/s]

106
341
0.2371364653243848
